# Introduction the the pairinteraction library

## Datatypes
The pairinteraction software supports different datatypes in its C++ backend implementation to adapt the performance and precision to the specific needs of the user.
The following datatypes are supported: `float`, `double`, `complexfloat`, `complexdouble`.
The `float` and `double` datatypes are used for real-valued calculations, while the `complexfloat` and `complexdouble` datatypes are used for complex-valued calculations, i.e. when using external fields in y-direction.
To change the datatype it is as simple as changing this import statement:

In [1]:
import os

os.environ["SPDLOG_LEVEL"] = "error"  # ignore infos and warnings from the C++ backend

import numpy as np
import pairinteraction.backend.double as pi  # possible backend data types: float, double, complexfloat, complexdouble

np.set_printoptions(linewidth=120, threshold=10)

## The Database object
The `Database` object is responsible for storing and looking up the allowed atomic states with their corresponding energies, quantum numbers and electric dipole, etc. matrix elements with other atomic states.
These matrix elements are pre-calculated (either via explicit calculation of the overlap integrals and using the Numerov method to get the radial wavefunctions, or alternatively for Earth Alkali atoms via Multichannel Quantum Defect Theory (MQDT)) and stored online in their own github repositories.
The `Database` object is able to download the necessary tables on the fly if `download_missing=True` is passed to the `Database`. Once downloaded, the tables are stored in the cache directory of the user's computer and are reused in subsequent calculations.

You can either create a `Database` object via `database = pi.Database(download_missing=True)` and use this database for the creation of the kets and basis objects below, or alternatively you can once create a global instance of the `Database` object via `pi.Database.create_global_instance(download_missing=True)` and then the ket and basis classes will use this global instance by default.

In [2]:
pi.Database.create_global_instance(download_missing=True)

# KetAtom
The simplest object you can create is a simple ket, e.g. via the `KetAtom` class.
The `KetAtom` object represents a single atomic state.
The first argument has to be the specifier of the atomic species.
Currently supported species are: ('Rb', 'Sr88_singlet', 'Sr88_triplet', 'Sr87_mqdt', 'Sr88_mqdt', 'Yb171_mqdt', 'Yb174_mqdt'), where no ending or '_singlet' or '_triplet' specifies that the matrix elements are calculated via the Numerov method, while '_mqdt' specifies that the matrix elements are calculated via MQDT.
The following optional keyword arguments of KetAtom have to uniquely specify the atomic state, you can pass whatever combination of quantum numbers you like, as long as they uniquely specify exactly one state (e.g. `pi.KetAtom("Rb", n=60, l=0, j=0.5, m=0.5)` and `pi.KetAtom("Rb", n=60, l=0, m=0.5)` are both equivalent and specify the same state).

In [3]:
ket = pi.KetAtom("Rb", n=60, l=1, j=0.5, m=0.5)

print(
    f"You successfully created a ket object: {ket} "
    f"with quantum numbers n={ket.n}, l={ket.l}, j={ket.j}, m={ket.m} and energy={ket.energy}"
)
print(f"And getting its energy in GHz is as simple as {ket.get_energy(unit='GHz')=}")

ket2 = pi.KetAtom("Rb", n=58, l=0, j=0.5, m=0.5)
delta_energy = ket.energy - ket2.energy  # this is a pint.Quantity object

print("Even the energy difference between two states is easily calculated and converted:")
print(f"delta_energy = {delta_energy}")
print(13 * " " + f"= {delta_energy.to('GHz', 'spectroscopy')} (as frequency)")
print(13 * " " + f"= {delta_energy.to('cm^-1', 'spectroscopy')} (as wavenumber)")
print(13 * " " + f"= {delta_energy.to('eV', 'spectroscopy')} ")
print(13 * " " + f"= {delta_energy.to('mm', 'spectroscopy')} (as wavelength)")

You successfully created a ket object: 60 P_{1/2}, m=1/2 with quantum numbers n=60, l=1.0, j=0.5, m=0.5 and energy=-0.0001520462348013062 hartree
And getting its energy in GHz is as simple as ket.get_energy(unit='GHz')=-1000.4161662949898
Even the energy difference between two states is easily calculated and converted:
delta_energy = 1.4033626284980249e-05 hartree
             = 92.33682521361543 gigahertz (as frequency)
             = 3.080024955584954 / centimeter (as wavenumber)
             = 0.00038187442527245385 electron_volt 
             = 3.2467269402694874 millimeter (as wavelength)


# BasisAtom
Next, you can create a basis object.
A basis object consists of a list of kets, which define a canonical basis for the Hilbert space.
Furthermore, the basis object defines basis states via its coefficients matrix, where each column in the coefficients matrix corresponds to one basis state.
When created the coefficients matrix is initialized to the identity matrix, i.e. each basis state correspond to one ket.
However, in general a state (and therefore each column of the basis coefficients matrix) can be a superposition of multiple kets.

The list of which kets should be considered in the basis can be restricted by passing in tuples of (min, max) values for the quantum numbers and the energy to the `BasisAtom` class.

In [4]:
energy_min, energy_max = ket.get_energy(unit="GHz") - 100, ket.get_energy(unit="GHz") + 100
basis = pi.BasisAtom("Rb", n=(58, 63), l=(0, 3), energy=(energy_min, energy_max), energy_unit="GHz")
coefficients = basis.coefficients  # this is a scipy.sparse.csr_matrix object

print(f"This basis contains {basis.number_of_kets} kets (=atomic states)")
print(f"The first and last kets are {basis.kets[0]} and {basis.kets[-1]}")
print()
print(f"The coefficient matrix has shape {coefficients.shape} and the following entries:")
print(f"{coefficients.toarray()}")

This basis contains 130 kets (=atomic states)
The first and last kets are 58 S_{1/2}, m=-1/2 and 63 P_{3/2}, m=3/2

The coefficient matrix has shape (130, 130) and the following entries:
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


# SystemAtom
The `SystemAtom` object describes the single atom system.
It is created by passing a `BasisAtom` object in, which defines the basis of the Hilbert space.
You can now set external fields and enable or disable the diamagnetic term.

Then you can inspect the resulting Hamiltonian and diagonalize it to get the eigenstates and eigenenergies of the system.

In [5]:
system = pi.SystemAtom(basis)
system.set_magnetic_field([0, 0, 1], unit="gauss")
system.set_electric_field([0.1, 0, 0.1], unit="V/cm")
system.enable_diamagnetism(True)

print("The Hamiltonian of the system (in GHz) with magnetic and electric fields is:")
print(f"{system.get_hamiltonian(unit='GHz').toarray()}")

The Hamiltonian of the system (in GHz) with magnetic and electric fields is:
[[-1.09290278e+03  0.00000000e+00  1.48688279e-01 ...  2.75998030e-03 -1.37999015e-03  0.00000000e+00]
 [ 0.00000000e+00 -1.09251874e+03 -1.48688279e-01 ...  1.37999015e-03  2.75998030e-03 -2.39021305e-03]
 [ 1.48688279e-01 -1.48688279e-01 -1.07404696e+03 ... -1.15289753e-04  0.00000000e+00  0.00000000e+00]
 ...
 [ 2.75998030e-03  1.37999015e-03 -1.15289753e-04 ... -9.03101897e+02  0.00000000e+00  0.00000000e+00]
 [-1.37999015e-03  2.75998030e-03  0.00000000e+00 ...  0.00000000e+00 -9.02846017e+02  0.00000000e+00]
 [ 0.00000000e+00 -2.39021305e-03  0.00000000e+00 ...  0.00000000e+00  0.00000000e+00 -9.02565408e+02]]


In [6]:
system.diagonalize()
eigenvalues = system.get_eigenvalues(unit="GHz")
print("The eigenenergies in GHz are:")
print(eigenvalues)

The eigenenergies in GHz are:
[-1092.90963606 -1092.52559537 -1074.06945565 ...  -903.06165295  -902.82597575  -902.55960746]


In [7]:
eigenbasis = system.get_eigenbasis()
ket = pi.KetAtom("Rb", n=60, l=1, j=0.5, m=0.5)
corresponding_state = eigenbasis.get_corresponding_state(ket)

eigenstate_number = 2
print(f"The eigenvector with index {eigenstate_number} is:")
print(f"{eigenbasis.coefficients.toarray()[:, eigenstate_number]}")
print(f"It has the largest overlap with the ket: {eigenbasis.get_corresponding_ket(eigenstate_number)}")


print(f"The state corresponding to the ket {ket} is:")
print(f"{corresponding_state.coefficients.toarray().flatten()}")
print(f"The overlap |<state|ket>|^2 is {corresponding_state.get_overlaps(ket)[0]}")

The eigenvector with index 2 is:
[ 1.00688253e-02 -6.63938226e-03  9.76009445e-01 ...  7.98604039e-07  5.12138625e-07 -2.48625426e-07]
It has the largest overlap with the ket: 58 P_{1/2}, m=-1/2
The state corresponding to the ket 60 P_{1/2}, m=1/2 is:
[ 8.42099547e-05  5.90292069e-05 -1.13201166e-06 ... -4.18636063e-06 -9.55374238e-06  1.86897257e-06]
The overlap |<state|ket>|^2 is 0.9774523600933283


# BasisCombined
The `BasisCombined` object consists of a list of `KetCombined` objects.
Again, we view these KetCombined kets as forming a canonical basis for the pair Hilbert space.
However, in contrast to the `KetAtom` objects, the `KetCombined` objects are not atomic states (or product states of atomic states), but rather product states of the eigenstates of the single atom Hamiltonian.

Again, the `BasisCombined` object has a coefficients matrix, which defines the basis states (with respect to the list of `KetCombined` objects).
The coefficients matrix is initialized to the identity matrix.
This corresponds to the eigenstates of the pair Hamiltonian with external fields but without any interaction between the atoms.
In general, when adding interactions, the pair-states (=the columns of the coefficent matrix) can be a superposition of multiple KetCombined objects.

Similar to the `BasisAtom` object, we can restrict the list of kets that should be considered in the basis by passing in tuples of (min, max) values for the energy (of the pair states) and the quantum number m if it is conserved.


In [8]:
pair_energy = 2 * pi.calculate_energy(ket, system, unit="GHz")
delta_energy = 3  # GHz
basis_combined = pi.BasisCombined(
    [system, system],
    energy=(pair_energy - delta_energy, pair_energy + delta_energy),
    energy_unit="GHz",
)
coefficients = basis_combined.coefficients

print(f"The combined basis contains {basis_combined.number_of_kets} KetCombined (=pair-states)")
print(f"The first KetCombined corresponds to a pair-state close to the product state {basis_combined.kets[0].label}")
print()
print(f"The coefficient matrix has shape {coefficients.shape} and the following entries:")
print(f"{coefficients.toarray()}")

The combined basis contains 470 KetCombined (=pair-states)
The first KetCombined corresponds to a pair-state close to the product state 58 P_{1/2}, m=-1/2; 61 D_{5/2}, m=-5/2

The coefficient matrix has shape (470, 470) and the following entries:
[[1. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 [0. 0. 1. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 1. 0. 0.]
 [0. 0. 0. ... 0. 1. 0.]
 [0. 0. 0. ... 0. 0. 1.]]


# SystemCombined
The `SystemCombined` object describes the pair system.
Similar to the `SystemAtom` object, it is created by passing a `BasisCombined` object in, which defines the basis of the pair Hilbert space.
You can now set the interatomic distance between the atoms and the order of the multipole expansion.
Then you can inspect the resulting Hamiltonian and diagonalize it to get the eigenstates and eigenenergies of the system.

In [9]:
system_combined = pi.SystemCombined(basis_combined)
distance = 5  # micrometer
system_combined.set_distance(distance, unit="micrometer")
system_combined.set_order(3)

print(f"The Hamiltonian of the SystemCombined (in GHz) for the given {distance=}um is:")
print(f"{system_combined.get_hamiltonian(unit='GHz').toarray()}")

The Hamiltonian of the SystemCombined (in GHz) for the given distance=5um is:
[[-1.99905344e+03  1.44436362e-06 -5.80274089e-07 ...  1.50597740e-13  1.66260152e-14 -1.52435325e-12]
 [ 1.44436362e-06 -1.99889679e+03  6.01147332e-07 ...  2.43956477e-12  9.83791932e-14  2.39416752e-11]
 [-5.80274089e-07  6.01147332e-07 -1.99872415e+03 ...  4.16073020e-12  5.04211447e-12  4.14304388e-10]
 ...
 [ 1.50597740e-13  2.43956477e-12  4.16073020e-12 ... -1.99790203e+03  4.58426350e-07 -1.43376618e-06]
 [ 1.66260152e-14  9.83791932e-14  5.04211447e-12 ...  4.58426350e-07 -1.99776208e+03 -6.58470419e-08]
 [-1.52435325e-12  2.39416752e-11  4.14304388e-10 ... -1.43376618e-06 -6.58470419e-08 -1.99767809e+03]]


In [10]:
system_combined.diagonalize()
eigenvalues = system_combined.get_eigenvalues(unit="GHz")
print("The eigenenergies in GHz are:")
print(eigenvalues)

The eigenenergies in GHz are:
[-2003.62340673 -2003.62338628 -2003.61450758 ... -1997.71626193 -1997.67805482 -1997.67805482]


# Calculate electric dipole matrix elements

In [11]:
pi.calculate_electric_dipole_matrix_element(ket, ket2, q=0)  # TODO: units! Coming soon ;)

55.06110733716566